In [1]:
import pyspark
import numpy as np
from pyspark.ml.feature import CountVectorizer, IDF, Tokenizer, HashingTF

In [2]:
def spark_session_setup():
    """
    creates a spark context
    >>> sc = spark_session_setup()
    """

    # in order to be bale to change log level
    conf = pyspark.SparkConf()
    conf.set('spark.logConf', 'true')
    conf.set('spark.executor.memory', '8G')
    conf.set('spark.driver.memory', '8G')
    conf.set('spark.driver.maxResultSize', '12G')

    # create a spark session
    sc = pyspark.SparkContext(appName='word_count', conf=conf)
#     sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

    # change log level to ERROR
    sc.setLogLevel("ERROR")
    return sc

spark_session = spark_session_setup()

In [3]:
sql_context = pyspark.sql.SQLContext(spark_session)

In [4]:
import re

In [5]:
filenames = open('../dataset/files/X_small_train.txt').read().split('\n')
labels = open('../dataset/files/y_small_train.txt').read().split('\n')
filename_label_dict = {}
for filename, label in zip(filenames, labels):
    filename_label_dict[filename] = label

broadcast_filename_label_dict = spark_session.broadcast(filename_label_dict)

In [6]:
rdd_files  = spark_session.wholeTextFiles("../dataset/data/train/")

In [7]:
rdd_files.take(1)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 1 times, most recent failure: Lost task 0.0 in stage 0.0 (TID 0, localhost, executor driver): java.net.SocketException: Connection reset by peer: socket write error
	at java.net.SocketOutputStream.socketWrite0(Native Method)
	at java.net.SocketOutputStream.socketWrite(SocketOutputStream.java:111)
	at java.net.SocketOutputStream.write(SocketOutputStream.java:155)
	at java.io.BufferedOutputStream.write(BufferedOutputStream.java:122)
	at java.io.DataOutputStream.write(DataOutputStream.java:107)
	at java.io.FilterOutputStream.write(FilterOutputStream.java:97)
	at org.apache.spark.api.python.PythonRDD$.writeUTF(PythonRDD.scala:393)
	at org.apache.spark.api.python.PythonRDD$.org$apache$spark$api$python$PythonRDD$$write$1(PythonRDD.scala:213)
	at org.apache.spark.api.python.PythonRDD$$anonfun$wri

In [ ]:
filter(lambda x: len(x)==2, ['as', 'de', 'sadas'])

In [ ]:
def pre_process(x):
    fname = x[0].split('/')[-1][:-6]
    label = int(broadcast_filename_label_dict.value[fname])
    word_list = list(filter(lambda x: len(x)==2 and x!='??', re.split('\r\n| ', x[1])))
    return (fname, label, word_list)

In [ ]:
bag_of_docs = rdd_files.map(pre_process)

In [ ]:
bag_of_docs.take(1)[0]

In [ ]:
doc_label_df = sql_context.createDataFrame(bag_of_docs, ['doc', 'label', 'text'])

In [ ]:
doc_label_df.show(1)

In [ ]:
# tokenizer = Tokenizer(inputCol="text", outputCol="words")

In [ ]:
# words_data = tokenizer.transform(doc_label_df)

In [ ]:
tf = HashingTF(inputCol="text", outputCol="counts")

In [ ]:
counts = tf.transform(doc_label_df)

In [ ]:
counts.show(1)

In [ ]:
idf = IDF(inputCol="counts", outputCol="features")

In [ ]:
idf_model = idf.fit(counts)

In [ ]:
rescaled_data = idf_model.transform(counts)

In [ ]:
rescaled_data.show(1)

In [ ]:
from pyspark.ml.classification import NaiveBayes
from pyspark.mllib.util import MLUtils
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
training, test = rescaled_data.randomSplit([0.6, 0.4])

In [ ]:
nb = NaiveBayes(smoothing=1.0, modelType="multinomial")

In [ ]:
model = nb.fit(training)

In [ ]:
predictions = model.transform(test)
predictions.show()

In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",
                                              metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test set accuracy = " + str(accuracy))